Imports

In [36]:
import numpy as np
import random
from pysat.solvers import Solver
from collections import deque


Task 1: Generate puzzles

1.1 define puzzle representations

using a 2D array , matrix grid where it is NxN grid size. 

(A-Z) is for the region labels
( * ) is for when a star is present in the grid


In [2]:
#example with 5x5 grid 1 star per row/column/region
puzzle = [
 ['A', 'B', 'C', 'C', 'C'],
 ['A', 'B', 'C', 'C', 'D'],
 ['A', 'A', 'C', 'D', 'D'],
 ['E', 'C', 'C', 'D', 'D'],
 ['E', 'E', 'E', 'D', 'D']
 ]

solution = [
 ['0', '1', '0', '0', '0'],
 ['0', '0', '0', '1', '0'],
 ['1', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '1'],
 ['0', '0', '1', '0', '0']
]
# example from puzzle-star-battle.com solved by myself

Similar for 10x10 grid with 2 stars per row/column/region, and even 14x14 grids with 3 stars per row/column/region

In [ ]:
print("example with labels")

1.2 Constraint representations

each row contains only k stars 
each column contains only k stars 
each region contains only k stars
no two stars can be adjacent

pre compute constraint maps

find values about puzzle

In [7]:
def get_star_limit(grid_size):
    if grid_size == 5:
        return 1
    elif grid_size == 6:
        return 1
    elif grid_size == 8:
        return 1
    elif grid_size == 10:
        return 2
    elif grid_size == 14:
        return 3
    else:
        raise ValueError(f"Grid size {grid_size} not supported")

Precompute and validate puzzles

In [19]:
def validate_solution(puzzle,solution):
    #get number of stars for grid size of puzzle
    grid_size =len(puzzle)
    star_lim = get_star_limit(grid_size)

    unique_regions = set(cell for row in puzzle for cell in row)

    # number of regions matches the grid size
    if len(unique_regions) != grid_size:
        return False

    #track star counts in row, column, region
    row_counts = np.zeros(len(puzzle), dtype = int)
    col_counts = np.zeros(len(puzzle[0]),dtype=int)
    region_counts = {region: 0 for region in set([item for sublist in puzzle for item in sublist])}

    # check puzzle and solution dimentions match
    if len(puzzle) != len(solution) or len(puzzle[0]) != len(solution):
        return False
    
    for r in range(grid_size):
        for c in range(grid_size):
            if solution[r][c] == '1':
                row_counts[r]+=1
                col_counts[c]+=1
                region = puzzle[r][c]

                #count stars in region
                if region not in region_counts:
                    region_counts[region]=0
                region_counts[region]+=1
    
    # check each row,colum,region has exactly n stars 
    if not (all(count == star_lim for count in row_counts) and
            all(count == star_lim for count in col_counts) and
            all(count == star_lim for count in region_counts.values())):
        return False
    
    #ensure stars dont touch diagnoally
    for r in range(grid_size):
        for c in range(grid_size):
            if solution[r][c] == '1':
                # check adjacent cells for stars
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        if dr == 0 and dc == 0:
                            continue
                        nr, nc = r + dr, c + dc
                        if 0 <= nr < len(puzzle) and 0 <= nc < len(puzzle[0]):
                            if solution[nr][nc] == '1':
                                return False
                            
    return True

In [13]:
#checking this works
valid = validate_solution(puzzle,solution)
print("solution?", valid)

solution? True


Some example puzzles to test 

In [17]:
#example puzzles 
puzzle_5x5 = [
    ['A', 'B', 'C', 'D', 'E'],
    ['A', 'B', 'C', 'D', 'F'],
    ['A', 'B', 'C', 'G', 'F'],
    ['A', 'H', 'C', 'G', 'F'],
    ['I', 'H', 'C', 'G', 'F']
]

solution_5x5 = [
    ['0', '0', '0', '1', '0'],
    ['0', '0', '1', '0', '0'],
    ['0', '0', '0', '0', '1'],
    ['1', '0', '0', '0', '0'],
    ['0', '1', '0', '0', '0']
]

puzzle_10x10 = [
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
]

solution_10x10 = [
    ['0', '1', '0', '0', '1', '0', '0', '0', '0', '0'],
    ['1', '0', '0', '0', '0', '1', '0', '0', '0', '0'],
    ['0', '0', '1', '0', '0', '0', '1', '0', '0', '0'],
    ['0', '1', '0', '0', '0', '0', '0', '1', '0', '0'],
    ['1', '0', '0', '1', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '1', '0', '1', '0', '0', '0'],
    ['0', '0', '1', '0', '0', '0', '0', '1', '0', '0'],
    ['1', '0', '0', '1', '0', '0', '0', '0', '0', '0'],
    ['0', '1', '0', '0', '0', '1', '0', '0', '0', '0'],
    ['0', '0', '1', '0', '0', '0', '0', '0', '1', '0']
]

puzzle_14x14 = [
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
]

solution_14x14 = [
    ['0', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0'],
    ['0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0'],
    ['1', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0'],
    ['1', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0'],
    ['0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '1', '0', '0', '0'],
    ['0', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0'],
    ['0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0'],
    ['1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0'],
    ['0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0'],
    ['1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0'],
    ['0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0'],
    ['0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0'],
    ['1', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0']
]


In [18]:
print(validate_solution(puzzle_5x5,solution_5x5),
      validate_solution(puzzle_10x10, solution_10x10),
      validate_solution(puzzle_14x14, solution_14x14))

False False False


1.3 Basic Generation Algorithm using sat solver

create regions

#ensure region integrity by shuffling labels by shuffling blocks of regions
# contiguous cells
# equally distributed no gaps or overlap  
# Regions must be connected orthogonally  
using floodfill algorithm to ensure contiguous regions

In [42]:
def generate_regions(grid_size):  
    regions = np.full((grid_size, grid_size), '', dtype=str)
    labels = [chr(65 + i) for i in range(grid_size)]
    random.shuffle(labels)

    def flood_fill(start_row, start_col, region_label):
        # up, down, left, right
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        
        # BFS
        queue = deque([(start_row, start_col)])
        regions[start_row][start_col] = region_label
        filled_cells = 1
        
        while queue:
            r, c = queue.popleft()
            # trys adding neighboring cells to the region
            random.shuffle(directions)  # random for varied filling
            for dr, dc in directions:
                nr, nc = r + dr, c + dc
                if 0 <= nr < grid_size and 0 <= nc < grid_size and regions[nr][nc] == '':
                    # Fill adjacent cells
                    regions[nr][nc] = region_label
                    queue.append((nr, nc))
                    filled_cells += 1
                    
                    if filled_cells == (grid_size * grid_size) // grid_size:  # Fill only enough cells per region
                        return
        
    # For each label, generate a region
    region_cells_needed = (grid_size * grid_size) // len(labels)
    
    for label in labels:
        # Randomly pick a start point in the grid
        start_row, start_col = random.randint(0, grid_size - 1), random.randint(0, grid_size - 1)
        
        # Ensure the starting point is not already assigned to a region
        while regions[start_row][start_col] != '':
            start_row, start_col = random.randint(0, grid_size - 1), random.randint(0, grid_size - 1)
        
        # Start flood-fill from this point to assign the region
        flood_fill(start_row, start_col, label)

    return regions

# # Example for testing
# grid_size = 5
# regions = generate_regions(grid_size)

# # Print the generated regions grid
# print(regions)


[['' '' '' 'C' 'C']
 ['' 'E' '' 'C' 'C']
 ['E' 'E' 'E' '' 'C']
 ['A' 'E' 'D' 'B' 'B']
 ['A' 'A' 'B' 'B' 'B']]


encode the star battle constraints into sat solver clauses so we can use sat solver (glucose3)

In [38]:
def encode_star_battle_sat(grid_size, n_stars, regions):
    """Encodes the Star Battle puzzle into a SAT problem."""
    solver = Solver(name="g3")  # Glucose3 SAT solver

    def var(r, c):
        """Generate a unique variable index for (r, c)."""
        return r * grid_size + c + 1  # SAT solvers use 1-based indexing

    # Row constraints: Each row must have exactly n_stars
    for r in range(grid_size):
        solver.add_clause([var(r, c) for c in range(grid_size)])  # At least one star in row
        solver.add_atmost([var(r, c) for c in range(grid_size)], n_stars)  # At most n_stars

    # Column constraints: Each column must have exactly n_stars
    for c in range(grid_size):
        solver.add_clause([var(r, c) for r in range(grid_size)])
        solver.add_atmost([var(r, c) for r in range(grid_size)], n_stars)

    # Region constraints: Each region must have exactly n_stars
    region_dict = {}
    for r in range(grid_size):
        for c in range(grid_size):
            region_dict.setdefault(regions[r][c], []).append(var(r, c))

    for region_vars in region_dict.values():
        solver.add_clause(region_vars)  # At least one star in each region
        solver.add_atmost(region_vars, n_stars)  # At most n_stars in region

    # Adjacency constraints: No two adjacent cells can have a star
    for r in range(grid_size):
        for c in range(grid_size):
            neighbors = [
                (r+dr, c+dc) for dr in [-1, 0, 1] for dc in [-1, 0, 1]
                if (dr != 0 or dc != 0) and 0 <= r+dr < grid_size and 0 <= c+dc < grid_size
            ]
            for nr, nc in neighbors:
                solver.add_clause([-var(r, c), -var(nr, nc)])  # If one cell has a star, the other cannot

    return solver



In [27]:
def generate_star_battle(grid_size):
    """Generates a Star Battle puzzle and ensures it has a unique solution using a SAT solver."""
    n_stars = 1 if grid_size == 5 else 2 if grid_size == 10 else 3
    regions = generate_regions(grid_size)

    # Encode constraints into a SAT problem
    solver = encode_star_battle_sat(grid_size, n_stars, regions)

    # Solve for a valid configuration
    if solver.solve():
        model = solver.get_model()
        solution = np.zeros((grid_size, grid_size), dtype=int)
        for r in range(grid_size):
            for c in range(grid_size):
                if model[r * grid_size + c] > 0:
                    solution[r][c] = 1  # Place star

        # Now check uniqueness by adding a **negation constraint** of the found solution
        negated_solution_clause = [-var(r, c) if solution[r][c] else var(r, c) for r in range(grid_size) for c in range(grid_size)]
        solver.add_clause(negated_solution_clause)

        # If solving again finds another solution, it's **not unique**, so we discard
        if solver.solve():
            return generate_star_battle(grid_size)  # Try again to generate a unique puzzle
        else:
            return regions, solution  # Unique solution found ✅

    return None, None



In [40]:
#test
grid_size = 5  

regions, solution = generate_star_battle(grid_size)

# Display the generated puzzle and solution
print("Generated Regions Grid:")
for row in regions:
    print(" ".join(row))

print("\nGenerated Solution Grid:")
for row in solution:
    print(" ".join(str(cell) for cell in row))


NotImplementedError: Atmost constraints are not supported by Glucose.

In [ ]:
#test
puzzle, solution = generate_star_battle(5)

print("Puzzle Grid:")
print(puzzle)

print("\nSolution Grid:")
print(solution)

1.3 Data structure choices

In [ ]:
print("examples class")

1.4 Storing and compiling puzzles

1.4 SAT solve to validate uniqueness of puzzles

1.6 Generate initial test set of puzzles

In [ ]:
print("test set of puzzles")